# <center>Crea tus propias funciones mágicas con IPython</center>

Vamos a crear una función mágica para obtener ayuda de la biblioteca estándar de Python fácilmente.

In [2]:
import requests
import re
from IPython.display import IFrame

Extraemos la lista de módulos de la [documentación de python](https://docs.python.org/3/py-modindex.html). Hay otras formas pero esto es un ejemplo conveniente para aprender.

In [5]:
mods = requests.get("https://docs.python.org/3.6/py-modindex.html").content

modules = re.findall(r'library/([^\'" >]+).html', str(mods))
modules

['__future__',
 '__main__',
 '_dummy_thread',
 '_thread',
 'abc',
 'aifc',
 'argparse',
 'array',
 'ast',
 'asynchat',
 'asyncio',
 'asyncore',
 'atexit',
 'audioop',
 'base64',
 'bdb',
 'binascii',
 'binhex',
 'bisect',
 'builtins',
 'bz2',
 'calendar',
 'cgi',
 'cgitb',
 'chunk',
 'cmath',
 'cmd',
 'code',
 'codecs',
 'codeop',
 'collections',
 'collections.abc',
 'colorsys',
 'compileall',
 'concurrent.futures',
 'configparser',
 'contextlib',
 'copy',
 'copyreg',
 'profile',
 'crypt',
 'csv',
 'ctypes',
 'curses',
 'curses.ascii',
 'curses.panel',
 'curses',
 'datetime',
 'dbm',
 'dbm',
 'dbm',
 'dbm',
 'decimal',
 'difflib',
 'dis',
 'distutils',
 'doctest',
 'dummy_threading',
 'email',
 'email.charset',
 'email.contentmanager',
 'email.encoders',
 'email.errors',
 'email.generator',
 'email.header',
 'email.headerregistry',
 'email.iterators',
 'email.message',
 'email.mime',
 'email.parser',
 'email.policy',
 'email.util',
 'codecs',
 'codecs',
 'codecs',
 'ensurepip',
 'enum',

Podemos mostrar la ayuda dentro de un IFrame:

In [6]:
url = "https://docs.python.org/3/library/{0}.html#module-{0}"
IFrame(url.format("__future__"), width = "100%", height = "400px")

### Estructura de una función mágica

> **De la documentación oficial de IPython: [definiendo funciones mágicas personalizadas](https://ipython.org/ipython-doc/3/config/custommagics.html)**

Existen dos maneras de crear una función mágica: 

* desde funciones autónomas
* heredando de una clase base provista por IPython: `IPython.core.magic.Magics`.

Las funciones mágicas pueden funcionar como:

* line magic (`from IPython.core.magic import register_line_magic`)
* cell magic (`from IPython.core.magic import register_cell_magic`)
* Ambos casos anteriores (`from IPython.core.magic import register_line_cell_magic`)

### 1) - El caso más simple, usando funciones y registrándolas

In [7]:
from IPython.core.magic import register_line_magic

# imports
import requests
import re
from IPython.display import IFrame, HTML

@register_line_magic
def stdlib_help(module):
    # list of modules
    mods = requests.get("https://docs.python.org/3/py-modindex.html")
    mods = mods.content
    modules = re.findall(r'library/([^\'" >]+).html', str(mods))
    
    # Check if argument is a stdlib module
    if module in modules:
        url = "https://docs.python.org/3/library/{0}.html#module-{0}"
        return IFrame(url.format(module), width = "100%", height = "400px")
    else:
        return HTML("<p><strong>"+module+"</strong> is not part of the CPython3.4 stdlib</p>")

In [8]:
%stdlib_help re

In [9]:
%stdlib_help regex

### 2) - Heredando de `IPython.core.magic.Magics`

Es muy similar al ejemplo anterior con algunos pasos más que añaden algo de funcionalidad extra: las funciones `magics` pueden potencialmente mantener estado entre llamadas y pueden tener acceso completo al objeto principal de IPython. En el caso actual no sería necesario así que la solución anterior sería suficiente.

In [ ]:
from IPython.core.magic import Magics, magics_class, line_magic

# imports
import requests
import re
from IPython.display import IFrame, HTML

@magics_class
class ItIsMagic(Magics):
    
    @register_line_magic
    def stdlib_help2(module):
        # list of modules
        mods = requests.get("https://docs.python.org/3/py-modindex.html")
        mods = mods.content
        modules = re.findall(r'library/([^\'" >]+).html', str(mods))

        # Check if argument is a stdlib module
        if module in modules:
            url = "https://docs.python.org/3/library/{0}.html#module-{0}"
            return IFrame(url.format(module), width = "100%", height = "400px")
        else:
            return HTML("<p><strong>"+module+"</strong> is not part of the CPython3.4 stdlib</p>")

# In order to actually use these magics, you must register them with a
# running IPython.  This code must be placed in a file that is loaded once
# IPython is up and running:
ip = get_ipython()
# You can register the class itself without instantiating it.  IPython will
# call the default constructor on it.
ip.register_magics(ItIsMagic)

In [ ]:
%stdlib_help2 __future__

In [ ]:
%stdlib_help2 make_the_web_I_have_in_mind

In [ ]:
from IPython.display import IFrame
from IPython.core.magic import register_line_magic

@register_line_magic
def ddg(arg):
    phrase = arg.replace(' ', '+')
    url = "https://duckduckgo.com/?&q={0}".format(phrase)
    print(url)
    return IFrame(url, "100%", 400)

In [ ]:
%ddg ipython

### Es fácil buscar nuevas posibilidades

<div class="alert alert-danger">
<p>Hay páginas que no son incrustables como google, stackoverflow...</p>

<P>Para comprobar si es posible mostrar una página como `iframe`, puedes usar el siguiente código:</p>
</div>

In [ ]:
import requests

g = requests.get("http://www.google.com")

g.headers['X-Frame-Options']

Si el resultado es `SAMEORIGIN` o `DENY` (puede haber otros) entonces no puedes empotrarla como `iframe`.

### Otros ejemplos explicados

- http://pybonacci.org/2014/01/24/157-cosas-de-ipython-que-no-sabias-y-nunca-preguntaste-i/ (in Spanish)
- http://nbviewer.ipython.org/urls/gist.github.com/bj0/5343292/raw/23a0845ee874827e3635edb0bf5701710a537bfc/jinja2.ipynb (in English)
- http://nbviewer.ipython.org/github/ipython-books/cookbook-code/blob/master/notebooks/chapter01_basic/04_magic.ipynb (in English, BTW, it is part of an excellent book "[IPython Interactive Computing and Visualization Cookbook](http://ipython-books.github.io/cookbook/)" written by Cyrille Rossant)

_______________________

###  Fuentes
>  - http://nbviewer.jupyter.org/github/kikocorreoso/PyConES14_talk-Hacking_the_Notebook/tree/master/notebooks/